# Set up the environment

Before running the notebook, please install the necessary packages and environment by running the following shell commands in your terminal:

```bash
# Create the conda environment from the provided environment file
conda env create -f ../conda_env_pkgs.yml -n soc_model_env

# Activate the new environment
conda activate soc_model_env

# Launch Jupyter Notebook (or JupyterLab) from within the environment
jupyter notebook


In [87]:
import json
import ee
import geemap
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score

# Authenticate and Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project= "ee-christopherharrellgis")

# Optional: Display map
#Map = geemap.Map(basemap = "SATELLITE")
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

#### Import SOC Samples and Study Area as Feature Collections

In [39]:
# Create a FeatureCollection from SOC_samples.csv
def create_feature_collection(df):
    features = []
    for idx, row in df.iterrows():
        point = ee.Geometry.Point([row['longitude'], row['latitude']])
        feature = ee.Feature(point, {
            'plot_no': row['plot_no'],
            'MgC_per_ha': row['MgC_per_ha'],
            'MgC_SE': row['MgC_SE']
        })
        features.append(feature)
    return ee.FeatureCollection(features)

# Load the study area GeoJSON file
study_area_geojson = '../soc/data/study_area.geojson'

# Load the SOC sample table
soc_samples_df = pd.read_csv("../soc/data/SOC_samples.csv")

with open(study_area_geojson) as f:
    geojson_data = json.load(f)

# Convert the study area to an Earth Engine FeatureCollection
study_area = ee.FeatureCollection(geojson_data)

soc_samples_points = create_feature_collection(soc_samples_df)

Map.addLayer(study_area, {'color': 'red'}, 'Study Area')
Map.addLayer(soc_samples_points, {'color': 'yellow'}, 'Sample Points')
Map.centerObject(study_area, zoom=11)

In [ ]:
""" wdpa = ee.FeatureCollection('WCMC/WDPA/current/polygons')

# Filter WDPA features that spatially intersect with your study area
overlapping_pas = wdpa.filter(ee.Filter.intersects('.geo', soc_samples_points.geometry()))

# Get the list of names
protected_areas_with_names = overlapping_pas.filter(ee.Filter.notNull(['NAME']))

# Get the names as a list
protected_area_names = protected_areas_with_names.aggregate_array('NAME')

# Print the names of protected areas that contain sample points
print('Protected Areas containing sample points:')
print(protected_area_names.getInfo())

Map.addLayer(overlapping_pas, {'color': 'blue'}, 'PAS') """

#### Generate ImageCollection for covariates

In [75]:
start_year = 2023
end_year = 2024

# COPERNICUS DEM (30m)
dem = ee.ImageCollection('COPERNICUS/DEM/GLO30').mosaic().select('DEM')

dem = dem.reproject(crs='EPSG:4326', scale=10)
slope = ee.Terrain.slope(dem)

# Visualization parameters for Elevation
vis_params_elevation = {
    'min': 0,
    'max': 3000,
    'palette': ['#00FFFF', '#0000FF', '#008000', '#FFFF00', '#FF0000', '#800000']
}

# Visualization parameters for slope
vis_params_slope = {
    'min': 0,
    'max': 60,
    #'palette': ['#00FF00', '#FFFF00', '#FFA500', '#FF0000']
}

Map.addLayer(dem.clip(study_area), vis_params_elevation, "DEM")
Map.addLayer(slope.clip(study_area), vis_params_slope, "slope")


In [66]:
terraclimate_precip = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE') \
    .filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
    .select('pr')
    
# Sum monthly precip per year
def annual_precip(year):
    year = ee.Number(year)
    year_images = terraclimate_precip.filter(ee.Filter.calendarRange(year, year, 'year'))
    annual = year_images.sum().set('year', year)
    return annual

years = ee.List.sequence(start_year, end_year)
annual_precip_collection = ee.ImageCollection(years.map(annual_precip))

# Mean of annual totals over 2 years
mean_annual_precip = annual_precip_collection.mean()

# Visualization for Precipitation
vis_params_precipitation = {
    'min': 0,
    'max': 2000,
    'palette': ['#f7fbff', '#6baed6', '#08306b']
}


In [71]:

terraclimate_temp = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE') \
    .filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
    .select(['tmmx', 'tmmn'])
    
def monthly_avg(img):
    tavg = img.expression(
        '(tmmx + tmmn) / 2',
        {
            'tmmx': img.select('tmmx'),
            'tmmn': img.select('tmmn')
        }
    ).rename('tavg')
    return tavg.copyProperties(img, img.propertyNames())

tavg_collection = terraclimate_temp.map(monthly_avg)

# Average across all months
mean_annual_temp = tavg_collection.mean()

# Visualization for Temperature
vis_params_temperature = {
    'min': -10,
    'max': 30,
    'palette': ['#313695', '#74add1', '#fdae61', '#a50026']
}

In [74]:
Map.addLayer(mean_annual_temp.clip(study_area), vis_params_temperature, 'Terra Temp')
Map.addLayer(mean_annual_precip.clip(study_area), vis_params_precipitation, 'Terra Precip')

In [89]:
def mask_s2_clouds(image):
    """Masks clouds and cirrus based on the QA60 band."""
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10  # Bit 10: clouds
    cirrus_bit_mask = 1 << 11  # Bit 11: cirrus
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
        qa.bitwiseAnd(cirrus_bit_mask).eq(0)
    )
    return image.updateMask(mask).divide(10000).copyProperties(image, image.propertyNames())

s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterDate(f'{start_year}-01-01', f'{end_year}-12-31') \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(mask_s2_clouds)

def calc_ndvi(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return ndvi.copyProperties(img, img.propertyNames())

def calc_evi(img):
    evi = img.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {
            'NIR': img.select('B8'),
            'RED': img.select('B4'),
            'BLUE': img.select('B2')
        }
    ).rename('EVI')
    return evi.copyProperties(img, img.propertyNames())

ndvi_collection = s2.map(calc_ndvi)
mean_ndvi = ndvi_collection.mean()

evi_collection = s2.map(calc_evi)
mean_evi = evi_collection.mean()

# EVI visualization
evi_vis = {
    'min': 0.0,
    'max': 1.0,
    'palette': ['purple', 'white', 'green']
}

# NDVI visualization
ndvi_vis = {
    'min': 0.0,
    'max': 1.0,
    'palette': ['blue', 'white', 'green']
}

Map.addLayer(mean_ndvi.clip(study_area), ndvi_vis, 'NDVI')
Map.addLayer(mean_evi.clip(study_area), evi_vis, 'EVI')

EEException: User memory limit exceeded.